In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# Import fast.ai libraries

In [ ]:
import fastai
from fastai.imports import *
from fastai.vision import *
from fastai.metrics import *
from fastai.gen_doc.nbdoc import *
print('fast.ai version:{}'.format(fastai.__version__))

In [ ]:
import os
WORK_DIR = os.getcwd()
WORK_DIR

In [ ]:
path = Path('../input/')
path.ls()

In [ ]:
path_img = path/'train'
fnames = get_image_files(path_img)
fnames[:5]

In [ ]:
np.random.seed(2)
pat = r"([a-z]+).\d+.jpg$"

In [ ]:
import re
prog = re.compile(pat)
result = prog.search('../input/train/dog.7504.jpg')
result[0]

In [ ]:
show_doc(get_transforms)

In [ ]:
batch_size = 64
tfms = get_transforms()
data = ImageDataBunch.from_name_re(path, fnames, pat, ds_tfms=tfms, test='test', size=224, bs=batch_size)
data = data.normalize(imagenet_stats)
data

In [ ]:
data.show_batch(rows=4, figsize=(7, 6))

In [ ]:
print(data.classes)
len(data.classes)

# Build the model

In [ ]:
model = models.resnet152
learn = cnn_learner(data, model, metrics=accuracy, model_dir=WORK_DIR)
learn.model

In [ ]:
learn.fit_one_cycle(2)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_top_losses(9, figsize=(15, 11))

In [ ]:
interp.plot_confusion_matrix(figsize=(4,4))

In [ ]:
learn.recorder.plot_losses()

In [ ]:
learn.recorder.plot_lr(show_moms=True)

In [ ]:
learn.save('stage-1')

In [ ]:
learn.unfreeze()
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.load('stage-1')
learn.unfreeze()
learn.fit_one_cycle(4, max_lr=slice(1e-6, 1e-4))

In [ ]:
learn.recorder.plot_losses()

In [ ]:
learn.save('stage-2')

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_top_losses(9, figsize=(15, 11))

In [ ]:
interp.plot_confusion_matrix(figsize=(4,4))

# predict

In [ ]:
try:
    # TTA cause error @fast.ai v1.0.39
    preds, _ = learn.TTA(ds_type=DatasetType.Test)
except:
    preds, _ = learn.get_preds(DatasetType.Test)
else:
    print('Predict with TTA done.')

In [ ]:
preds

In [ ]:
# from here we know that 'cats' is label 0 and 'dogs' is label 1.
print(data.classes)
dict_label_order = {label:order for order,label in enumerate(data.classes)}
print(dict_label_order)

In [ ]:
n_dogs = dict_label_order['dog']
n_dogs

In [ ]:
prob_dogs = preds[:,n_dogs].numpy()
prob_dogs

In [ ]:
plt.hist(prob_dogs)

In [ ]:
ids = [int(file.stem) for file in data.test_ds.x.items]
ids[:10]

In [ ]:
import pandas as pd
submission = pd.DataFrame({'id':ids,'label':prob_dogs})
submission = submission.sort_values(by=['id'])

In [ ]:
submission.head()

In [ ]:
submission.label.describe()

In [ ]:
submission.to_csv('submission.csv', index=False)